In [2]:
%load_ext autoreload
%autoreload 2
#class variables are initialized in config
from config import *
import pandas as pd
import numpy as np
from datetime import datetime
import time
import logging
import os
from emailscraper_app.modules.Sending_Emails.sends import EmailConfig
from emailscraper_app.modules.Sending_Emails.sends import SendMail
from emailscraper_app.modules.Sending_Emails.html_email_strings.schools_sport_focus import get_template
#This import dictates template


#create log dir and file
logpath_creation = os.getcwd() + '\\Logs'
if not os.path.exists(logpath_creation):
    os.makedirs(logpath_creation)

logging.basicConfig(filename= logpath_creation + '\\Email_Sender.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S', force=True)


df = pd.read_csv('phony.csv')
# ---------------------------

def blast(email_config, df, test=False):


    # Get the next 50 emails iteratively and send every 60 seconds with a new SMTP connection
    # get_next_50 is what ends this function. It will break once end_point has surpassed.
    while len(df) >= 1:

        next_50 = SendMail.get_next_50(df)
        try:
            new_point = next_50.index[0]
            end_point = next_50.index[-1]   
            print(new_point, end_point) 
            logging.info(f'\n\nStarting email send at index {new_point}, and ending at index {end_point}')
        except IndexError:
            print('List has been fully iterated through')

         # Check if the end_point surpasses 1500
        if end_point >= 1500:
            logging.info('Reached the end_point limit of 1500. Exited the loop.')
            print('Reached the end_point limit of 1500. Exited the loop.')
            break
        
        # Process the emails
        email_history = SendMail.process(next_50, email_config, test)
        email_history.to_csv('output.csv', index = False)

        
        if test:
            logging.info('Breaking loop after sending to personal emails due to test')
            print('Breaking loop after sending to personal emails due to test')
            break  # Terminate after one iteration if test=True
        
        interval_seconds = 30
        # Wait for the specified interval before the next iteration
        try:
            time.sleep(interval_seconds)
            logging.info(f'Sleeping for {interval_seconds} seconds completed')
            print(f'Sleeping for {interval_seconds} seconds')
        except:
            logging.info('Issue with the sleep')
            print('Issue with the sleep')
    
    os.remove('output.csv')

#If test ARG is True it will keep sending to personal email
#& output.csv must not exist at the beginning of new runs
blast(email_config, df, test=True)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Output is not created, first run of 50 emails preparing to be sent
0 2
Test argument is True, cutting down frame and sending to personal emails
0 High School A
1 High School B
Breaking loop after sending to personal emails due to test


In [11]:
from emailscraper_app.modules.Sending_Emails import KC_schools

df = KC_schools.read_in()
df = KC_schools.filter_emails_by_sport(df, ['Baseball', 'Softball'])

In [16]:
print('Here are the columns\n', df.columns)

Here are the columns
 Index(['SchoolDistrict', 'Level', 'HighSchools', 'Position', 'email'], dtype='object')
